In [151]:
import pandas as pd
import sqlite3 as sq
import SJreader_functions as SJ
import numpy as np

conn = sq.connect('SJreader.db')

In [164]:
key_list = raw_input("Enter up to 5 keywords, separated by commas:")

Enter up to 5 keywords, separated by commas:mouse, brain, tumor, gene, therapy


In [165]:
keys = key_list.split(",")
keys = [x.strip().lower() for x in keys]

In [166]:
user_keys = list(set(keys))
user_keys = [unicode(x, "utf-8") for x in user_keys]

In [167]:
article_info = pd.read_sql('SELECT * FROM recent_articles;', conn, index_col = 'id')
titles = article_info['title'].values
journals = article_info['journal'].values
abstracts = article_info['abstract'].values
url = article_info['url'].values

In [168]:
title_key_scores = SJ.title_key_scores(user_keys, titles)
keyword_scores = SJ.keyword_scores(user_keys, article_info)
abstract_key_scores = SJ.abstract_key_scores(user_keys, abstracts)


In [169]:
match_list = []
check_match_list = []
yes_list = []
no_list = []

for N in range(len(title_key_scores)):
    if (title_key_scores[N]+keyword_scores[N]+abstract_key_scores[N]) >7:
        match_list.append(N)
    elif (title_key_scores[N]+keyword_scores[N]+abstract_key_scores[N]) >4:
        check_match_list.append(N)

if len(match_list)>1:

    print("Found %i top matches" %len(match_list))
    for m in match_list:
        print('\n'+titles[m]+'\n')
        keep = raw_input("Are you interested in this article? (Y/N, A to see abstract, or quit)")
        if keep.lower() == 'quit':
            break 
        if keep.lower() == 'a':
            print(abstracts[m]+'\n')
            keep = raw_input("Are you interested in this article? (Y/N or quit)")
        if keep.lower() == 'y':
            yes_list.append(m)
            print("You can see the full article at:")
            print(url[m]+'\n')
        elif keep.lower() == 'n':
            no_list.append(m)

elif len(check_match_list)>1:
    print("Found %i possible matches" %len(check_match_list))
    for m in check_match_list:
        print('\n'+titles[m]+'\n')
        keep = raw_input("Are you interested in this article? (Y/N, A to see abstract, or quit)")
        if keep.lower() == 'quit':
            break    
        if keep.lower() == 'a':
            print(abstracts[m]+'\n')
            keep = raw_input("Are you interested in this article? (Y/N or quit)")
        if keep.lower() == 'y':
            yes_list.append(m)
            print("You can see the full article at:")
            print(url[m]+'\n')
        elif keep.lower() == 'n':
            no_list.append(m)
        
else: print("No matches found")
    
print("Done")

Found 32 possible matches

Mitochondrial changes and oxidative stress in a mouse model of Zellweger syndrome neuropathogenesis

Are you interested in this article? (Y/N, A to see abstract, or quit)n

Differential effects of peripheral and brain tumor necrosis factor on inflammation, sickness, emotional behavior and memory in mice

Are you interested in this article? (Y/N, A to see abstract, or quit)y
You can see the full article at:
http://www.sciencedirect.com/science/article/pii/S0889159116303580


Circadian Rhythm Disruption Promotes Lung Tumorigenesis

Are you interested in this article? (Y/N, A to see abstract, or quit)y
You can see the full article at:
http://www.sciencedirect.com/science/article/pii/S1550413116303126


Chemotherapy enhances tumor vascularization via Notch signaling-mediated formation of tumor-derived endothelium in breast cancer

Are you interested in this article? (Y/N, A to see abstract, or quit)n

Carbon ion radiotherapy decreases the impact of tumor heteroge

In [170]:
user_articles = article_info.iloc[yes_list+no_list]

yes_list_plus1 = [x+1 for x in yes_list]
n_matches = len(yes_list+no_list)
user_articles.insert(0,'match', np.zeros(n_matches), allow_duplicates = False)
user_articles.loc[yes_list_plus1,'match'] = 1


In [171]:
user_articles[['match','title','journal']]

,match,title,journal
id,,,
1077,1,Differential effects of peripheral and brain t...,"Brain, Behavior, and Immunity"
2093,1,Circadian Rhythm Disruption Promotes Lung Tumo...,Cell Metabolism
2242,1,Carbon ion radiotherapy decreases the impact o...,Cancer Letters
2646,1,"Non-invasive, neuron-specific gene therapy by ...",Journal of Controlled Release
950,0,Mitochondrial changes and oxidative stress in ...,Neuroscience
2108,0,Chemotherapy enhances tumor vascularization vi...,Biochemical Pharmacology
